## Open the dataset:

In [7]:
import pandas as pd

df = pd.read_json( 'Musical_Instruments_5.json',lines=True)

df.sample(10)


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
7764,A1BD73IAW04XMZ,B003AYK8XW,victormanuel3007,"[0, 0]","Great output Jack, i love this one much more b...",5,Fender 1/4&#34; Output Jack,1358294400,"01 16, 2013"
8927,A2Y2U697SS98E1,B004Z17008,"John Wade Long Jr. ""LongJnSilver""","[0, 0]",The Snark is great. I have 2 for my 4 ukuleles...,5,A+++,1398297600,"04 24, 2014"
2680,A1SD1C8XK3Z3V1,B0002GYYDQ,"guitfiddleblue ""guitfiddleblue""","[0, 0]",The digitech main squeeze is an okay compressi...,3,ok,1386633600,"12 10, 2013"
4849,A31B0LYT2C92SN,B000K69V5E,Victor Hugo,"[0, 0]",I'm using this pedal like a clean boost for le...,5,Great pedal,1355788800,"12 18, 2012"
1230,ABBE2DL4IFLM5,B0002E1G5C,"Andrew D. Flint ""Andrew D Flint""","[0, 0]",I won't rant about a string winder.. They are...,4,It just works,1398988800,"05 2, 2014"
2616,AG3PVU7DCXZTK,B0002GXZK4,"Coffee Infusion ""coffeeinfusion""","[1, 1]","This was my first acoustic guitar, and I've re...",5,Wonderful acoustic at a price that's hard to b...,1351987200,"11 4, 2012"
6857,AXX6BZDD8K4JL,B001PGXKC8,Eric,"[6, 7]",These cords seem to be a hair longer than the ...,3,Nice addition to my pedal board,1346803200,"09 5, 2012"
2356,A2CA4YBYSETM02,B0002GLDQM,Operafanatic,"[0, 0]",I have a black composite acoustic guitar and t...,5,Stays on my guitar,1357430400,"01 6, 2013"
7701,A1FJBWP2R3NSH4,B0038O9NLM,LJS,"[1, 1]",This is a very high quality strap. The supple ...,3,Very Nice Strap -- But Not For Me,1358726400,"01 21, 2013"
10176,AEKID8YBJSV8K,B00FXKIG5I,"Gary Jennings ""JenningsMusic""","[0, 0]",I love these clip-on tuners. I have several bu...,3,Hard to see angle,1392681600,"02 18, 2014"


## Get the Bing Liu lexicon

In [5]:
from sklearn import preprocessing

import nltk

nltk.download('opinion_lexicon')

from nltk.corpus import opinion_lexicon

from nltk.tokenize import word_tokenize

print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))

print('Examples of positive words in opinion lexicon',

     opinion_lexicon.positive()[:10])

print('Examples of negative words in opinion lexicon',

     opinion_lexicon.negative()[:10])


Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/peter/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


## Create the “Dictionary”

In [6]:
#Let's create a dictionary which we can use for scoring our review text

nltk.download('punkt')

df.rename(columns={"reviewText": "text"}, inplace=True)

pos_score = 1

neg_score = -1

word_dict = {}


# Adding the positive words to the dictionary

for word in opinion_lexicon.positive():

     word_dict[word] = pos_score


# Adding the negative words to the dictionary

for word in opinion_lexicon.negative():

     word_dict[word] = neg_score


[nltk_data] Downloading package punkt to /Users/peter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Function to do the math

In [19]:
def bing_liu_score(text):

    sentiment_score = 0

    bag_of_words = word_tokenize(text.lower())

    for word in bag_of_words:

         if word in word_dict:

               sentiment_score += word_dict[word]

    return sentiment_score


## Make sure all texts are non-empty and are type text then call the function

In [16]:
df['text'].fillna('no review', inplace=True)

df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)

## Let’s see the output

In [17]:
df[['overall',"text", 'Bing_Liu_Score']].head(10)

,overall,text,Bing_Liu_Score
0,4,Contains some interesting stitches.,1
1,5,I'm a fairly experienced knitter of the one-co...,22
2,4,Great book but the index is terrible. Had to w...,0
3,5,I purchased the Kindle edition which is incred...,4
4,5,Very well laid out and very easy to read.\n\nT...,5
5,5,"Beginning her career as a freelance knitter, M...",15
6,5,This is a terrific stitch handbook (and I have...,9
7,4,The book needs to be coil bound. The content i...,1
8,5,I really am enjoying this book! I like the siz...,12
9,5,Just received this book and looked over it cov...,6


In [18]:
df.groupby('overall').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
overall,
1,-0.255049
2,0.566098
3,1.158796
4,2.028146
5,2.130005
